In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import zipfile
import os
import pandas as pd

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/environment recognition/places365_contents.zip'

# Directory to extract the contents
extracted_dir = 'data_contents/'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Create a list to store dataframe rows
data = []

# Traverse through the extracted directory to create the dataframe
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            label_name = os.path.basename(root)
            data.append({'path': file_path, 'name': file, 'label': label_name})

# Create a DataFrame
df = pd.DataFrame(data)

# Adjust path format for 'path name' column
df['path name'] = 'data_contents/' + df['label'] + '/' + df['name']

# Rearrange columns
df = df[['path name', 'name', 'label']]

# Print the DataFrame
print(df)

                                      path name          name          label
0      data_contents/garage-indoor/00003219.jpg  00003219.jpg  garage-indoor
1      data_contents/garage-indoor/00000329.jpg  00000329.jpg  garage-indoor
2      data_contents/garage-indoor/00003871.jpg  00003871.jpg  garage-indoor
3      data_contents/garage-indoor/00001855.jpg  00001855.jpg  garage-indoor
4      data_contents/garage-indoor/00000039.jpg  00000039.jpg  garage-indoor
...                                         ...           ...            ...
62515    data_contents/art_gallery/00001223.jpg  00001223.jpg    art_gallery
62516    data_contents/art_gallery/00000348.jpg  00000348.jpg    art_gallery
62517    data_contents/art_gallery/00001421.jpg  00001421.jpg    art_gallery
62518    data_contents/art_gallery/00003544.jpg  00003544.jpg    art_gallery
62519    data_contents/art_gallery/00004748.jpg  00004748.jpg    art_gallery

[62520 rows x 3 columns]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'df'
# Split the DataFrame into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Data augmentation: random rotations
    width_shift_range=0.2,  # Data augmentation: random width shifts
    height_shift_range=0.2,  # Data augmentation: random height shifts
    horizontal_flip=True,  # Data augmentation: horizontal flips
)
batch_size = 32
# Load and preprocess training data from DataFrame
train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='path name',  # Column containing image paths
    y_col='label',  # Column containing labels
    target_size=(112, 112),  # Resize images to match model input size
    batch_size=batch_size,
    class_mode='categorical',  # Categorical labels for multi-class classification
)

# Load and preprocess validation data from DataFrame
validation_generator = datagen.flow_from_dataframe(
    val_df,
    x_col='path name',
    y_col='label',
    target_size=(112, 112),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 50016 validated image filenames belonging to 13 classes.
Found 12504 validated image filenames belonging to 13 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/environment recognition/capsnet_model_100_epochs.h5')

In [ ]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

# # Save the model
# model.save('/content/drive/MyDrive/envirnment recognition/model_200_epochs.h5')  # Adjust the filename as needed

391/391 [==============================] - 135s 343ms/step - loss: 11.9709 - accuracy: 0.1526
Validation Loss: 11.970903396606445
Validation Accuracy: 0.15259116888046265
